In [1]:
import os 
os.chdir("../")
%pwd

'd:\\Programming\\ML\\End-to-End\\CV\\End-to-End-PlantVillage'

In [2]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass 
class PrepareBaseModelSchema:
    root_dir: Path 
    base_model_path: Path 
    updated_base_model_path: Path
    params_image_size : list 
    params_learning_rate: float 
    params_weights: str 
    params_classes: int
    params_include_top: bool


In [3]:
from src.PlantVillage.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.PlantVillage.utils import create_directories, read_yaml
class ConfigurationManager:
    def __init__(self,
                config = CONFIG_FILE_PATH,
                params = PARAMS_FILE_PATH):
        self.config = read_yaml(config)
        self.params = read_yaml(params)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_base_model_config(self) -> PrepareBaseModelSchema:
        config = self.config.prepare_base_model 
        params = self.params 
        create_directories([config.root_dir])
        
        return PrepareBaseModelSchema(
            root_dir= config.root_dir,
            base_model_path= config.base_model_path,
            updated_base_model_path= config.updated_base_model_path,
            params_image_size= params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_weights=params.WEIGHTS,
            params_classes=params.CLASSES,
            params_include_top=params.INCLUDE_TOP
        )

In [4]:
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelSchema):
        self.config = config 
        
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )
        
        self.model.save(self.config.base_model_path)
        
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
                
        flatten = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation= "softmax"
        )(flatten)
        
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs= prediction
        )
        
        full_model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics= ["accuracy"]
        )
        full_model.summary()
        return full_model 
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all=True, 
            freeze_till= None,
            learning_rate= self.config.params_learning_rate
        )
        
        self.full_model.save(self.config.updated_base_model_path)

In [5]:
try:
    config = ConfigurationManager() 
    prepare_base_model_config = config.get_base_model_config()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.get_base_model() 
    prepare_base_model.update_base_model()
    
    
    
except Exception as e:
    raise e

[2026-01-26 17:28:38,167][INFO][__init__] -> yaml file: config\config.yaml loaded Successfully.
[2026-01-26 17:28:38,169][INFO][__init__] -> yaml file: params.yaml loaded Successfully.
[2026-01-26 17:28:38,171][INFO][__init__] -> Directory created Successfully at: artifacts
[2026-01-26 17:28:38,173][INFO][__init__] -> Directory created Successfully at: artifacts/prepare_base_model


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │       376,335 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,091,023 (57.57 MB)

 Trainable params: 376,335 (1.44 MB)

 Non-trainable params: 14,714,688 (56.13 MB)